Reads and parses fNIRS data with traditional ML techniques

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os

In [2]:
s2_fnirs_path = os.path.join(os.getcwd(), "data/S902/2015-02-26_11-24-48-120", "fNIRSdata.txt")
s2_marker_path = os.path.join(os.getcwd(), "data/S902/2015-02-26_11-24-48-120", "markers.txt")

In [11]:
s2_fnirs_df = pd.read_csv(s2_fnirs_path, sep='\t', lineterminator='\r', skiprows=[0,1,2,3],
                          dtype={"Matlab_now": float,
                                 "Increasing Marker": float,
                                 "BoxyTime": float,
                                 "Marker": float,
                                 "A-DC1": float,
                                 "A-DC2": float,
                                 "A-DC3": float,
                                 "A-DC4": float,
                                 "A-DC5": float,
                                 "A-DC6": float,
                                 "A-DC7": float,
                                 "A-DC8": float,
                                 "B-DC1": float,
                                 "B-DC2": float,
                                 "B-DC3": float,
                                 "B-DC4": float,
                                 "B-DC5": float,
                                 "B-DC6": float,
                                 "B-DC7": float,
                                 "B-DC8": float},
                         index_col=False)


/Users/sjjin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
s2_marker_df = pd.read_csv(s2_marker_path, sep='\t', lineterminator='\r', skiprows=[0,1,2,3],
                           index_col=False,
                         dtype={"NbPoints": int,
                                "Matlab_now": float,
                                "BoxyTime": float,
                                "Simulus_String": str,
                                "Tral_no": int,
                                "Stimulus_label": float})

In [14]:
s2_fnirs_df

,Matlab_now,Increasing Marker,BoxyTime,Marker,A-DC1,A-DC2,A-DC3,A-DC4,A-DC5,A-DC6,A-DC7,A-DC8,B-DC1,B-DC2,B-DC3,B-DC4,B-DC5,B-DC6,B-DC7,B-DC8
0,41088.8,0.0,5943.069,0.0,1954.0,492.9,135.9,41.25,1846.0,452.4,112.5,29.92,287.2,79.62,24.38,7.141,385.5,117.5,37.50,13.12
1,41089,0.0,5943.153,0.0,1959.0,489.0,134.9,41.34,1847.0,453.0,112.9,29.88,285.5,79.85,24.46,7.150,384.8,117.5,37.39,13.11
2,41089.1,0.0,5943.237,0.0,1964.0,491.5,135.8,41.23,1846.0,455.3,113.4,29.91,286.0,79.82,24.55,7.153,385.1,117.4,37.29,13.09
3,41089.1,0.0,5943.321,0.0,1952.0,494.6,135.3,41.15,1846.0,453.5,113.6,29.97,286.5,79.79,24.56,7.136,385.0,117.2,37.51,13.15
4,41089.3,0.0,5943.447,0.0,1965.0,493.2,135.1,41.36,1850.0,453.2,113.1,29.94,286.6,79.57,24.58,7.157,386.2,117.1,37.57,13.16
5,41089.3,0.0,5943.531,0.0,1952.0,494.8,135.7,41.41,1839.0,454.1,113.5,30.01,286.3,79.77,24.64,7.159,386.0,118.3,37.78,13.21
6,41089.4,0.0,5943.615,0.0,1955.0,496.2,136.5,41.37,1842.0,455.3,113.9,30.04,288.3,79.84,24.56,7.188,386.7,117.7,37.63,13.27
7,41089.5,0.0,5943.678,0.0,1963.0,493.6,136.0,41.47,1837.0,455.8,113.5,30.18,290.0,80.06,24.58,7.191,387.9,118.6,37.91,13.28
8,41089.6,0.0,5943.763,0.0,1962.0,499.2,136.3,41.74,1848.0,457.8,113.5,30.03,285.8,80.15,24.76,7.208,388.9,117.8,38.00,13.28
9,41089.7,0.0,5943.847,0.0,1955.0,494.8,136.0,41.43,1837.0,455.6,113.7,29.92,287.9,79.96,24.66,7.202,387.0,117.9,37.69,13.16


In [15]:
s2_marker_df

,NbPoints,Matlab_now,BoxyTime,Stimulus_String,Trial_no,Stimulus_Label,Stimulus_Time
0,\n342,41117.786,5971.909,1;baselinestart;1424967917714;,1,baselinestart,1.424968e+12
1,\n1037,41176.770,6030.871,1;baselineend;1424967976760;,1,baselineend,1.424968e+12
2,\n1112,41183.101,6037.234,7;easystart;1424967983086;,7,easystart,1.424968e+12
3,\n1466,41213.115,6067.251,7;easyend;1424968013103;,7,easyend,1.424968e+12
4,\n1774,41239.258,6093.380,22;hardstart;1424968039235;,22,hardstart,1.424968e+12
5,\n2128,41269.257,6123.418,22;hardend;1424968069238;,22,hardend,1.424968e+12
6,\n2436,41295.370,6149.527,5;easystart;1424968095285;,5,easystart,1.424968e+12
7,\n2789,41325.289,6179.459,5;easyend;1424968125288;,5,easyend,1.424968e+12
8,\n3097,41351.417,6205.611,4;easystart;1424968151388;,4,easystart,1.424968e+12
9,\n3451,41381.447,6235.626,4;easyend;1424968181391;,4,easyend,1.424968e+12
